<a href="https://colab.research.google.com/github/waytogoasis/app/blob/main/pso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# 1. Acesso à Planilha e Autenticação (Mantido)
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

# Abra a planilha pelo URL
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/17PjZc1KP6RVSjKsSRhUmzfMX225DiQxSg1N2_UISyLA'  # Substitua pelo seu URL
try:
    spreadsheet = gc.open_by_url(spreadsheet_url)
    worksheet = spreadsheet.sheet1  # Acessa a primeira aba da planilha
except gspread.exceptions.SpreadsheetNotFound:
    print(f"Planilha não encontrada no URL: {spreadsheet_url}. Verifique o URL e as permissões de compartilhamento.")
    exit()

# 2. Leitura dos Dados da Planilha (Mantido)

data = worksheet.get_all_values()
header = data[0]
rows = data[1:]

# Cria um dicionário para armazenar os dados de cada turbina
turbines_data = {}
for row in rows:
    segment_turbine = row[0]
    segment = int(segment_turbine[0])
    turbine = segment_turbine[1]
    if turbine not in turbines_data:
        turbines_data[turbine] = {}
    if segment not in turbines_data[turbine]:
        turbines_data[turbine][segment] = {}

    turbines_data[turbine][segment] = {
        'intensities': [int(val) for val in row[1:]]
    }

# 3. Implementação do Algoritmo PSO

import random
import numpy as np

def pso_optimization(turbine_data, num_particles=10, iterations=50, w=0.7, c1=1.5, c2=1.5):
    """
    Algoritmo PSO para otimizar a intensidade das bobinas.

    Args:
        turbine_data (dict): Dicionário contendo os dados da turbina.
        num_particles (int): Número de partículas no enxame.
        iterations (int): Número de iterações do algoritmo.
        w (float): Fator de inércia.
        c1 (float): Coeficiente cognitivo.
        c2 (float): Coeficiente social.

    Returns:
        dict: Dicionário contendo o melhor padrão de excitação encontrado.
    """

    def fitness_function(intensities):
        return sum(intensities) + intensities[3] * 0.5 + intensities[4] * 1

    best_global_intensities = [0] * 5
    best_global_fitness = -1

    particles = []
    for _ in range(num_particles):
        particle = {
            'intensities': [random.randint(0, 10) for _ in range(5)],
            'best_intensities': [0] * 5,
            'best_fitness': -1,
            'velocity': [0] * 5
        }
        particles.append(particle)

    for iteration in range(iterations):
        for particle in particles:
            fitness = fitness_function(particle['intensities'])

            if fitness > particle['best_fitness']:
                particle['best_fitness'] = fitness
                particle['best_intensities'] = particle['intensities'][:]

            if fitness > best_global_fitness:
                best_global_fitness = fitness
                best_global_intensities = particle['intensities'][:]

            for j in range(5):
                r1, r2 = random.random(), random.random()
                cognitive_component = c1 * r1 * (particle['best_intensities'][j] - particle['intensities'][j])
                social_component = c2 * r2 * (best_global_intensities[j] - particle['intensities'][j])
                particle['velocity'][j] = w * particle['velocity'][j] + cognitive_component + social_component
                particle['intensities'][j] = int(max(0, min(10, particle['intensities'][j] + particle['velocity'][j])))

        print(f"Iteração {iteration+1}: Melhor Global = {best_global_intensities}, Aptidão = {best_global_fitness:.2f}")

    return {'best_intensities': best_global_intensities, 'best_fitness': best_global_fitness}

# 4. Execução do PSO para cada Turbina

optimized_turbines = {}
for turbine, segments in turbines_data.items():
    print(f"Otimizando a turbina {turbine}:")
    optimized_intensities = {}
    for segment, data in segments.items():
        print(f" Otimizando para o segmento {segment}:")
        best_solution = pso_optimization(data, num_particles=20, iterations=100)
        optimized_intensities[segment] = best_solution['best_intensities']
    optimized_turbines[turbine] = optimized_intensities

# 5. Geração de Gráficos com Estilo e Salvamento na Pasta "pso"

import matplotlib.pyplot as plt
from IPython.display import Image, display
from google.colab import drive
import os

# Montar o Google Drive
drive.mount('/content/drive')

# Definir o caminho para a pasta "pso" dentro da pasta "graficos"
pso_graficos_path = '/content/drive/My Drive/graficos/pso'

# Criar a pasta "pso" se ela não existir
if not os.path.exists(pso_graficos_path):
    os.makedirs(pso_graficos_path)

# Definir o estilo dos gráficos
plt.style.use('dark_background')
neon_colors = ['#00FF00', '#00FFFF', '#FF9900', '#FF00FF', '#FFFF00']

def generate_visualizations_pso(optimized_turbines, original_data):
    """
    Gera visualizações gráficas dos resultados do PSO e as salva na pasta "pso".

    Args:
        optimized_turbines (dict): Dicionário com os resultados otimizados do PSO.
        original_data (list): Dados originais da planilha.
    """

    original_intensities = {}
    for row in original_data[1:]:
        segment_turbine, *intensities = row
        turbine = segment_turbine[1]
        segment = int(segment_turbine[0])
        if turbine not in original_intensities:
            original_intensities[turbine] = {}
        original_intensities[turbine][segment] = [int(val) for val in intensities]

    # Gráfico 1
    plt.figure(figsize=(12, 6))
    for i, turbine in enumerate(sorted(optimized_turbines.keys())):
        original_avg = [np.mean(original_intensities[turbine][seg]) for seg in sorted(original_intensities[turbine].keys())]
        optimized_avg = [np.mean(optimized_turbines[turbine][seg]) for seg in sorted(optimized_turbines[turbine].keys())]
        plt.plot(original_avg, label=f'Original - Turbina {turbine}', color=neon_colors[i%5], linestyle='--')
        plt.plot(optimized_avg, label=f'Otimizado - Turbina {turbine}', color=neon_colors[i%5])
    plt.xlabel('Segmento')
    plt.ylabel('Intensidade Média')
    plt.title('Comparação das Intensidades Médias por Turbina (PSO)')
    plt.legend()
    plt.savefig(f'{pso_graficos_path}/grafico_1.png')
    plt.close()
    display(Image(filename=f'{pso_graficos_path}/grafico_1.png'))

    # Gráfico 2
    plt.figure(figsize=(10, 5))
    segments = sorted(optimized_turbines['a'].keys())
    intensities = [optimized_turbines['a'][seg] for seg in segments]
    plt.bar(segments, [np.mean(i) for i in intensities], color=neon_colors[0])
    plt.xlabel('Segmento')
    plt.ylabel('Intensidade Média')
    plt.title('Intensidades Otimizadas por Segmento (Turbina a) - PSO')
    plt.savefig(f'{pso_graficos_path}/grafico_2.png')
    plt.close()
    display(Image(filename=f'{pso_graficos_path}/grafico_2.png'))

    # Gráfico 3
    plt.figure(figsize=(10, 5))
    original_b = [np.mean(original_intensities['b'][seg]) for seg in sorted(original_intensities['b'].keys())]
    optimized_b = [np.mean(optimized_turbines['b'][seg]) for seg in sorted(optimized_turbines['b'].keys())]
    plt.plot(original_b, label='Original', color=neon_colors[1], linestyle='--')
    plt.plot(optimized_b, label='Otimizado', color=neon_colors[1])
    plt.xlabel('Segmento')
    plt.ylabel('Intensidade Média')
    plt.title('Comparação das Intensidades por Segmento (Turbina b) - PSO')
    plt.legend()
    plt.savefig(f'{pso_graficos_path}/grafico_3.png')
    plt.close()
    display(Image(filename=f'{pso_graficos_path}/grafico_3.png'))

    # Gráfico 4
    plt.figure(figsize=(8, 6))
    all_intensities = [[np.mean(optimized_turbines[turbine][seg]) for seg in sorted(optimized_turbines[turbine].keys())] for turbine in sorted(optimized_turbines.keys())]
    plt.imshow(all_intensities, cmap='magma', aspect='auto')
    plt.colorbar(label='Intensidade Média')
    plt.yticks(np.arange(len(optimized_turbines.keys())), labels=sorted(optimized_turbines.keys()))
    plt.xticks(np.arange(5), labels=['1', '2', '3', '4', '5'])
    plt.xlabel('Segmento')
    plt.ylabel('Turbina')
    plt.title('Mapa de Calor das Intensidades Otimizadas - PSO')
    plt.savefig(f'{pso_graficos_path}/grafico_4.png')
    plt.close()
    display(Image(filename=f'{pso_graficos_path}/grafico_4.png'))

    # Gráfico 5
    fig, axs = plt.subplots(2, 3, figsize=(18, 10), sharey=True)
    fig.suptitle('Dispersão entre Intensidade Média e Segmento para cada Turbina - PSO')
    for i, turbine in enumerate(sorted(optimized_turbines.keys())):
        row, col = i // 3, i % 3
        intensities = [np.mean(optimized_turbines[turbine][seg]) for seg in sorted(optimized_turbines[turbine].keys())]
        axs[row, col].scatter(range(1, 6), intensities, color=neon_colors[i%5])
        axs[row, col].set_title(f'Turbina {turbine}')
        axs[row, col].set_xlabel('Segmento')
        axs[row, col].set_ylabel('Intensidade Média')
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.savefig(f'{pso_graficos_path}/grafico_5.png')
    plt.close()
    display(Image(filename=f'{pso_graficos_path}/grafico_5.png'))

    # Gráfico 6
    plt.figure(figsize=(8, 5))
    intensities_e = [item for sublist in [optimized_turbines['e'][seg] for seg in sorted(optimized_turbines['e'].keys())] for item in sublist]
    plt.hist(intensities_e, bins=10, color=neon_colors[4], edgecolor='black')
    plt.xlabel('Intensidade')
    plt.ylabel('Frequência')
    plt.title('Histograma das Intensidades para a Turbina e - PSO')
    plt.savefig(f'{pso_graficos_path}/grafico_6.png')
    plt.close()
    display(Image(filename=f'{pso_graficos_path}/grafico_6.png'))

    # Gráfico 7
    plt.figure(figsize=(10, 6))
    plt.boxplot(list(map(list, zip(*all_intensities))), labels=['1', '2', '3', '4', '5'], patch_artist=True,
                boxprops=dict(facecolor=neon_colors[2], color='white'),
                capprops=dict(color='white'),
                whiskerprops=dict(color='white'),
                flierprops=dict(color='white', markeredgecolor='white'),
                medianprops=dict(color='white'))
    plt.xlabel('Segmento')
    plt.ylabel('Intensidade')
    plt.title('Boxplot das Intensidades por Segmento - PSO')
    plt.savefig(f'{pso_graficos_path}/grafico_7.png')
    plt.close()
    display(Image(filename=f'{pso_graficos_path}/grafico_7.png'))

    # Gráfico 8
    plt.figure(figsize=(12, 6))
    for i, turbine in enumerate(sorted(optimized_turbines.keys())):
        intensities = [np.mean(optimized_turbines[turbine][seg]) for seg in sorted(optimized_turbines[turbine].keys())]
        plt.plot(range(1, 6), intensities, marker='o', color=neon_colors[i%5], label=f'Turbina {turbine}')
    plt.xlabel('Segmento')
    plt.ylabel('Intensidade Média')
    plt.title('Intensidades por Segmento para cada Turbina - PSO')
    plt.xticks(range(1, 6))
    plt.legend()
    plt.savefig(f'{pso_graficos_path}/grafico_8.png')
    plt.close()
    display(Image(filename=f'{pso_graficos_path}/grafico_8.png'))

    # Gráfico 9
    num_vars = 5
    angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
    angles += angles[:1]
    fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
    for i, turbine in enumerate(sorted(optimized_turbines.keys())):
        intensities = [np.mean(optimized_turbines[turbine][seg]) for seg in sorted(optimized_turbines[turbine].keys())]
        intensities += intensities[:1]
        ax.plot(angles, intensities, linewidth=1, linestyle='solid', color=neon_colors[i%5], label=f"Turbina {turbine}")
        ax.fill(angles, intensities, color=neon_colors[i%5], alpha=0.25)
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)
    plt.xticks(angles[:-1], ['Segmento 1', '2', '3', '4', '5'])
    ax.set_rlabel_position(0)
    plt.yticks([2, 5, 8], ["2", "5", "8"], color="grey", size=7)
    plt.ylim(0, 10)
    plt.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
    plt.title('Gráfico de Radar para as Intensidades Médias de cada Turbina - PSO', size=11, y=1.1)
    plt.savefig(f'{pso_graficos_path}/grafico_9.png')
    plt.close()
    display(Image(filename=f'{pso_graficos_path}/grafico_9.png'))


Otimizando a turbina a:
 Otimizando para o segmento 1:
Iteração 1: Melhor Global = [10, 2, 6, 9, 9], Aptidão = 49.50
Iteração 2: Melhor Global = [10, 8, 6, 9, 9], Aptidão = 55.50
Iteração 3: Melhor Global = [10, 6, 6, 10, 10], Aptidão = 57.00
Iteração 4: Melhor Global = [10, 7, 6, 10, 10], Aptidão = 58.00
Iteração 5: Melhor Global = [9, 9, 10, 10, 10], Aptidão = 63.00
Iteração 6: Melhor Global = [10, 10, 10, 10, 10], Aptidão = 65.00
Iteração 7: Melhor Global = [10, 10, 10, 10, 10], Aptidão = 65.00
Iteração 8: Melhor Global = [10, 10, 10, 10, 10], Aptidão = 65.00
Iteração 9: Melhor Global = [10, 10, 10, 10, 10], Aptidão = 65.00
Iteração 10: Melhor Global = [10, 10, 10, 10, 10], Aptidão = 65.00
Iteração 11: Melhor Global = [10, 10, 10, 10, 10], Aptidão = 65.00
Iteração 12: Melhor Global = [10, 10, 10, 10, 10], Aptidão = 65.00
Iteração 13: Melhor Global = [10, 10, 10, 10, 10], Aptidão = 65.00
Iteração 14: Melhor Global = [10, 10, 10, 10, 10], Aptidão = 65.00
Iteração 15: Melhor Global = [